In [ ]:
#Creating a CNN model with cifar100 dataset
#Import Libraries
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.models import Sequential

In [ ]:
#Load data and normalize image data
(X_train, y_train), (X_test, y_test) = cifar100.load_data()

X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [ ]:
# Name of all classes in CIFAR-100
classes = ['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 
           'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 
           'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'computer_keyboard', 
           'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 
           'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 
           'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 
           'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 
           'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 'turtle', 
           'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman', 'worm']

In [ ]:
#visualization
plt.figure(figsize=(20, 3))

for i in range(10):
    plt.subplot(1, 10, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_test[i], cmap="gray")
    plt.xlabel(classes[y_test[i].astype(int)[0]])
    
plt.show()

In [ ]:
#Encoding label data
'''
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
y_train=enc.fit_transform(y_train).toarray().astype(int)
y_test=enc.transform(y_test).toarray().astype(int)


print(y_train.shape)
print(y_train[0])
'''
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, 100)
y_test = to_categorical(y_test, 100)

In [ ]:
#the process of enlarging images from 32x32 to 64x64 size

#X_train = np.array([tf.image.resize(img, (64,64)) for img in X_train])
#X_test = np.array([tf.image.resize(img, (64,64)) for img in X_test])

#Smart_resize is selected due to the amount of RAM.
X_train = np.array([tf.keras.preprocessing.image.smart_resize(img, (64,64)) for img in X_train])
X_test = np.array([tf.keras.preprocessing.image.smart_resize(img, (64,64)) for img in X_test])

In [ ]:
#the process of creating validation data (10% of the test data was used)
from sklearn.model_selection import train_test_split

X_test, X_val, y_test, y_val = train_test_split(X_train, y_train, test_size=0.5, random_state=30)

In [ ]:
#Generate augmented images
datagen = ImageDataGenerator(
            rotation_range = 20, #Rotation up to 20 degrees.
            width_shift_range = 0.2, #20% horizontal scrolling.
            height_shift_range = 0.2, #20% vertical scrolling.
            shear_range = 0.2, #20% scrolling on the image.
            zoom_range = 0.2, #20% zoom in-out on the image.
            horizontal_flip = True, #inverting an image horizontally.
            fill_mode = 'nearest' #The process of filling empty areas with the closest pixels.
)
datagen.fit(X_train)

In [ ]:
#Build CNN model

#First feature extraction layer
model = Sequential()
model.add(Conv2D(32,(3,3), padding='same', input_shape = (64,64,3)))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Conv2D(32,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size = (2,2)))

#Second feature extraction layer
model.add(Conv2D(32,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Conv2D(64,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Conv2D(64,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size = (2,2)))

#Third feature extraction layer
model.add(Conv2D(64,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Conv2D(128,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Conv2D(128,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size = (2,2)))

#Fourth feature extraction layer
model.add(Conv2D(256,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Conv2D(256,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Conv2D(256,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(GlobalAveragePooling2D())

#Classification layer
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dense(256))
model.add(BatchNormalization(momentum = 0.95))
model.add(LeakyReLU())
model.add(Dense(100, activation='softmax'))

model.summary()

In [ ]:
#Model Compile
model.compile(optimizer=Adam(decay = 1e-6),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
#Model fit with early stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=7, restore_best_weights=True)
history = model.fit(datagen.flow(X_train, y_train, batch_size = 256),
                    validation_data = (X_val, y_val),
                    epochs = 100,
                    callbacks = [early_stopping])

In [ ]:
#Evaluating the model with test data
loss, acc = model.evaluate(X_test, y_test, verbose=1)

In [ ]:
#the process of plotting accuracy and loss using a line graph
plt.figure()
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], marker = 'o', label = 'Training Accuracy')
plt.plot(history.history['val_accuracy'], marker = 'x', label = 'Validation Accuracy')
plt.title('Acc')
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.grid(True)
plt.legend()

plt.figure()
plt.subplot(1,2,2)
plt.plot(history.history['loss'], marker = 'o', label = 'Training Loss')
plt.plot(history.history['val_loss'], marker = 'x', label = 'Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
#Model save
model.save('cifar100_CNN.keras')

In [ ]:
#Load model
from tensorflow.keras.models import load_model
model = load_model('cifar100_CNN.keras')

In [ ]:
#Prediction with test dataset
y_pred = model.predict(X_test)

In [ ]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix
prediction = []
true_labels = []

for i in range(X_test.shape[0]):
  prediction.append(np.argmax(y_pred[i]))
  true_labels.append(np.argmax(y_test[i]))

cm = confusion_matrix(prediction, true_labels)
print(cm)

In [ ]:
# Calculating f1 score
from sklearn.metrics import f1_score
print(f"f1 score: {f1_score(true_labels, prediction, average='weighted')}")

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(true_labels, prediction, target_names=classes, digits=5))

In [ ]:
# Plot the confusion matrix
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(24,24))
ax = fig.add_subplot(211)
cax = ax.matshow(cm)
plt.title('Confusion matrix of the classifier')
fig.colorbar(cax)
ax.set_xticklabels([''] + classes)
ax.set_yticklabels([''] + classes)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()